In [1]:
import gym
import random

In [2]:
env = gym.make('Taxi-v3').env
env.reset()


352

In [30]:
print(env.action_space)
print(env.observation_space)
len(env.P)


Discrete(6)
Discrete(500)


500

In [31]:
a = env.render(mode='ansi')
print(a)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


